In [ ]:
import random
import torch
import gc
from typing import Dict, List, Tuple

def algoritmo_genetico_lstm(
        serie_temporal: torch.Tensor,
        tamano_poblacion: int = 4,
        generaciones: int = 2,
        max_muestras: int = 100_000
    ) -> Dict[str, float]:
    """Implementa un algoritmo genético para optimizar hiperparámetros de una red LSTM.

    Parámetros:
        serie_temporal (torch.Tensor): Serie temporal de entrada para entrenamiento
        tamano_poblacion (int): Número de individuos por generación (default 4)
        generaciones (int): Número de iteraciones evolutivas (default 2)
        max_muestras (int): Límite de muestras para evitar saturación de RAM (default 100,000)

    Retorna:
        Dict[str, float]: Mejor conjunto de hiperparámetros encontrado
    """

    def crear_individuo() -> Dict[str, float]:
        """Genera un individuo con hiperparámetros aleatorios."""
        individuo = {
            'longitud_secuencia': random.randint(10, 50),
            'tamano_capa_oculta': random.choice([16, 32, 64, 128]),
            'num_capas': random.choice([1, 2, 3]),
            'dropout': random.uniform(0.0, 0.5),
            'tasa_aprendizaje': random.uniform(0.0005, 0.01)
        }
        # Ajuste para redes con una sola capa
        if individuo['num_capas'] == 1:
            individuo['dropout'] = 0.0
        return individuo

    def evaluar_aptitud(individuo: Dict[str, float]) -> Tuple[float, Dict[str, float]]:
        """Evalúa un individuo mediante el entrenamiento del modelo LSTM."""
        # Preparación de datos
        muestra = serie_temporal[:max_muestras]
        X, y = crear_secuencias(muestra, individuo['longitud_secuencia'])

        if len(X) < 500:
            return float('inf'), individuo  # Penaliza secuencias muy cortas

        # División train/validation
        X_ent, X_val, y_ent, y_val = dividir_conjuntos(X, y, prueba=0.2, mezclar=False)

        # Conversión a tensores
        X_ent = torch.tensor(X_ent, dtype=torch.float32).unsqueeze(-1)
        y_ent = torch.tensor(y_ent, dtype=torch.float32).view(-1, 1)
        X_val = torch.tensor(X_val, dtype=torch.float32).unsqueeze(-1)
        y_val = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)

        # Entrenamiento del modelo
        modelo = LSTM_PreciosBitcoin(
            entrada=1,
            oculto=individuo['tamano_capa_oculta'],
            capas=individuo['num_capas'],
            dropout=individuo['dropout']
        )

        perdida = entrenar_modelo(
            modelo=modelo,
            X_ent=X_ent,
            y_ent=y_ent,
            X_val=X_val,
            y_val=y_val,
            epocas=2,
            lr=individuo['tasa_aprendizaje']
        )

        # Liberación de memoria
        del modelo, X_ent, y_ent, X_val, y_val
        torch.cuda.empty_cache()
        gc.collect()

        return perdida, individuo

    def cruzar_individuos(
            padre1: Dict[str, float],
            padre2: Dict[str, float]
        ) -> Dict[str, float]:
        """Realiza cruce y mutación para crear un nuevo individuo."""
        hijo = {
            'longitud_secuencia': random.choice([padre1['longitud_secuencia'], padre2['longitud_secuencia']]),
            'tamano_capa_oculta': random.choice([padre1['tamano_capa_oculta'], padre2['tamano_capa_oculta']]),
            'num_capas': random.choice([padre1['num_capas'], padre2['num_capas']]),
            'dropout': max(0.0, min(0.5, (padre1['dropout'] + padre2['dropout']) / 2 + random.uniform(-0.05, 0.05))),
            'tasa_aprendizaje': max(0.0001, min(0.01, (padre1['tasa_aprendizaje'] + padre2['tasa_aprendizaje']) / 2 + random.uniform(-0.001, 0.001)))
        }

        if hijo['num_capas'] == 1:
            hijo['dropout'] = 0.0

        return hijo

    # Inicialización de población
    poblacion = [crear_individuo() for _ in range(tamano_poblacion)]

    # Ciclo evolutivo
    for generacion in range(1, generaciones + 1):
        # Evaluación de aptitud
        resultados = []
        for individuo in poblacion:
            perdida, ind = evaluar_aptitud(individuo)
            resultados.append((perdida, ind))

        # Validación de resultados
        if len(resultados) < 2:
            print(" Advertencia: Insuficientes individuos válidos")
            break

        # Selección de mejores
        resultados.sort(key=lambda x: x[0])
        mejores = resultados[:max(2, tamano_poblacion // 2)]
        nueva_poblacion = [ind for _, ind in mejores]

        # Reproducción
        while len(nueva_poblacion) < tamano_poblacion:
            if len(nueva_poblacion) >= 2:
                padre1, padre2 = random.sample(nueva_poblacion, 2)
            else:
                padre1 = padre2 = nueva_poblacion[0]

            nuevo_individuo = cruzar_individuos(padre1, padre2)
            nueva_poblacion.append(nuevo_individuo)

        poblacion = nueva_poblacion
        print(f"🧬 Generación {generacion}/{generaciones} | Mejor pérdida: {mejores[0][0]:.6f}")

    return resultados[0][1] if resultados else crear_individuo()